In [11]:
# Import libraries and datasets

import pandas as pd
import sys, os

sys.path.append(os.path.abspath('../'))  ## sets directory to parent directory

climate = pd.read_csv('../data/sample_climate_data.csv', parse_dates=['date'])
claims = pd.read_csv('../data/synthetic_claims.csv', parse_dates=['date'])

# Compute rolling averages 
climate['temperature_ma_3d'] = climate['temperature_c'].rolling(window=3).mean()
climate['precipitation_ma_3d'] = climate['precipitation_mm'].rolling(window=3).mean()



In [12]:
# Run the claim logic on climate data

from src.parametric_triggers import detect_streak_trigger

climate = detect_streak_trigger(
    climate, 
    column = 'temperature_ma_3d',
    threshold = 30, 
    min_streak=3
)

In [14]:
# Run the claims logic on claims data

from src.claim_classifier import classify_claims

claims = classify_claims(claims)

In [19]:
# Merge claims and climate

merged = pd.merge(
    claims,
    climate[['date', 'location', 'above_threshold', 'triggered']],
    on=['date', 'location'],
    how='left'
)

merged = merged.rename(columns={'triggered': 'climate_triggered'})

In [22]:
# Combine to create insurance decision

merged['insurance_decision'] = merged['payout_triggered'] & merged['climate_triggered']



,claim_id,location,date,claim_text,payout_triggered,above_threshold,climate_triggered,insurance_decision
0,1,Farm_A,2023-07-07,We experienced extreme rainfall for three days...,True,True,True,True
1,2,Farm_B,2023-07-04,The weather has been hot but manageable — no c...,False,False,False,False
2,3,Farm_C,2023-07-06,"We had five days of temperatures above 35°C, a...",True,True,True,True
3,4,Farm_A,2023-07-01,Just a brief shower — nothing serious.,False,False,False,False
4,5,Farm_B,2023-07-05,"The rainfall exceeded our irrigation capacity,...",True,False,False,False


,claim_id,location,date,claim_text,payout_triggered,above_threshold,climate_triggered,insurance_decision
0,1,Farm_A,2023-07-07,We experienced extreme rainfall for three days...,True,True,True,True
1,2,Farm_B,2023-07-04,The weather has been hot but manageable — no c...,False,False,False,False
2,3,Farm_C,2023-07-06,"We had five days of temperatures above 35°C, a...",True,True,True,True
3,4,Farm_A,2023-07-01,Just a brief shower — nothing serious.,False,False,False,False
4,5,Farm_B,2023-07-05,"The rainfall exceeded our irrigation capacity,...",True,False,False,False
